In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from pokemon_card_generator.data import card_data

cards_df = card_data.get_card_data()

In [3]:
cards_df.head()

,id,name,subtypes,hp,types,evolvesFrom,abilities,attacks,weaknesses,convertedRetreatCost,rarity,flavorText,evolvesTo,resistances,rules,releaseDate
0,base1-1,Alakazam,Stage 2,80.0,Psychic,Kadabra,"[{'name': 'Damage Swap', 'text': 'As often as ...","[{'name': 'Confuse Ray', 'cost': ['Psychic', '...",Psychic,3.0,Rare Holo,Its brain can outperform a supercomputer. Its ...,NaN,NaN,NaN,1999-01-09
1,base1-2,Blastoise,Stage 2,100.0,Water,Wartortle,"[{'name': 'Rain Dance', 'text': 'As often as y...","[{'name': 'Hydro Pump', 'cost': ['Water', 'Wat...",Lightning,3.0,Rare Holo,A brutal Pokémon with pressurized water jets o...,NaN,NaN,NaN,1999-01-09
2,base1-3,Chansey,Basic,120.0,Colorless,NaN,NaN,"[{'name': 'Scrunch', 'cost': ['Colorless', 'Co...",Fighting,1.0,Rare Holo,A rare and elusive Pokémon that is said to bri...,[Blissey],Psychic,NaN,1999-01-09
3,base1-4,Charizard,Stage 2,120.0,Fire,Charmeleon,"[{'name': 'Energy Burn', 'text': 'As often as ...","[{'name': 'Fire Spin', 'cost': ['Fire', 'Fire'...",Water,3.0,Rare Holo,Spits fire that is hot enough to melt boulders...,NaN,Fighting,NaN,1999-01-09
4,base1-5,Clefairy,Basic,40.0,Colorless,NaN,NaN,"[{'name': 'Sing', 'cost': ['Colorless'], 'conv...",Fighting,1.0,Rare Holo,Its magical and cute appeal has many admirers....,[Clefable],Psychic,NaN,1999-01-09


In [14]:
cards_df[cards_df.attacks.isna()].head()

,id,name,subtypes,hp,types,evolvesFrom,abilities,attacks,weaknesses,convertedRetreatCost,rarity,flavorText,evolvesTo,resistances,rules,releaseDate
221,base3-3,Ditto,Basic,50.0,Colorless,NaN,"[{'name': 'Transform', 'text': 'If Ditto is yo...",NaN,Water,1.0,Rare Holo,Capable of copying an enemy's genetic code to ...,NaN,Psychic,NaN,1999-10-10
236,base3-18,Ditto,Basic,50.0,Colorless,NaN,"[{'name': 'Transform', 'text': 'If Ditto is yo...",NaN,Fighting,1.0,Rare,Capable of copying an enemy's genetic code to ...,NaN,Psychic,NaN,1999-10-10
779,neo1-22,Elekid,Baby,30.0,Lightning,NaN,"[{'name': 'Playful Punch', 'text': 'Once durin...",NaN,Water,NaN,Rare,"It rotates its arms to generate electricity, b...",[Electabuzz],NaN,[If this Baby Pokémon is your Active Pokémon a...,2000-12-16
908,neo2-40,Igglybuff,Baby,30.0,Colorless,Jigglypuff,"[{'name': 'Gaze', 'text': 'Once during your tu...",NaN,Grass,NaN,Uncommon,"It has a very soft body. If it starts to roll,...",[Jigglypuff],NaN,[If this Baby Pokémon is your Active Pokémon a...,2001-06-01
1676,ecard3-47,Buried Fossil,Basic,30.0,Colorless,NaN,"[{'name': 'Reconstruction', 'text': 'Once duri...",NaN,Fire,1.0,Common,NaN,NaN,NaN,"[Evolves into Omanyte, Kabuto, Aerodactyl, You...",2003-05-12


In [15]:
[item for item in cards_df.attacks]

[[{'name': 'Confuse Ray',
   'cost': ['Psychic', 'Psychic', 'Psychic'],
   'convertedEnergyCost': 3,
   'damage': '30',
   'text': 'Flip a coin. If heads, the Defending Pokémon is now Confused.'}],
 [{'name': 'Hydro Pump',
   'cost': ['Water', 'Water', 'Water'],
   'convertedEnergyCost': 3,
   'damage': '40+',
   'text': "Does 40 damage plus 10 more damage for each Water Energy attached to Blastoise but not used to pay for this attack's Energy cost. Extra Water Energy after the 2nd doesn't count."}],
 [{'name': 'Scrunch',
   'cost': ['Colorless', 'Colorless'],
   'convertedEnergyCost': 2,
   'damage': '',
   'text': "Flip a coin. If heads, prevent all damage done to Chansey during your opponent's next turn. (Any other effects of attacks still happen.)"},
  {'name': 'Double-edge',
   'cost': ['Colorless', 'Colorless', 'Colorless', 'Colorless'],
   'convertedEnergyCost': 4,
   'damage': '80',
   'text': 'Chansey does 80 damage to itself.'}],
 [{'name': 'Fire Spin',
   'cost': ['Fire', 'F

In [24]:
attack1_name = [
    item[0]['name'] if isinstance(item, list) else item
    for item in cards_df.attacks
]

attack1_cost = [
    item[0]['cost'] if isinstance(item, list) else item
    for item in cards_df.attacks
]

attack1_converted_cost = [
    len(cost) if isinstance(cost, list) else cost for cost in attack1_cost
]

attack1_damage = [
    item[0]['damage'] if isinstance(item, list) else item
    for item in cards_df.attacks
]

attack1_text = [
    item[0]['text'] if isinstance(item, list) else item
    for item in cards_df.attacks
]

In [44]:
cards_df['attack1_name'] = attack1_name
cards_df['attack1_cost'] = attack1_cost
cards_df['attack1_converted_cost'] = attack1_converted_cost
cards_df['attack1_damage'] = attack1_damage
cards_df['attack1_text'] = attack1_text

In [45]:
cards_df.head()

,id,name,subtypes,hp,types,evolvesFrom,abilities,attacks,weaknesses,convertedRetreatCost,...,flavorText,evolvesTo,resistances,rules,releaseDate,attack1_name,attack1_cost,attack1_damage,attack1_text,attack1_converted_cost
0,base1-1,Alakazam,Stage 2,80.0,Psychic,Kadabra,"[{'name': 'Damage Swap', 'text': 'As often as ...","[{'name': 'Confuse Ray', 'cost': ['Psychic', '...",Psychic,3.0,...,Its brain can outperform a supercomputer. Its ...,NaN,NaN,NaN,1999-01-09,Confuse Ray,"[Psychic, Psychic, Psychic]",30,"Flip a coin. If heads, the Defending Pokémon i...",3.0
1,base1-2,Blastoise,Stage 2,100.0,Water,Wartortle,"[{'name': 'Rain Dance', 'text': 'As often as y...","[{'name': 'Hydro Pump', 'cost': ['Water', 'Wat...",Lightning,3.0,...,A brutal Pokémon with pressurized water jets o...,NaN,NaN,NaN,1999-01-09,Hydro Pump,"[Water, Water, Water]",40+,Does 40 damage plus 10 more damage for each Wa...,3.0
2,base1-3,Chansey,Basic,120.0,Colorless,NaN,NaN,"[{'name': 'Scrunch', 'cost': ['Colorless', 'Co...",Fighting,1.0,...,A rare and elusive Pokémon that is said to bri...,[Blissey],Psychic,NaN,1999-01-09,Scrunch,"[Colorless, Colorless]",,"Flip a coin. If heads, prevent all damage done...",2.0
3,base1-4,Charizard,Stage 2,120.0,Fire,Charmeleon,"[{'name': 'Energy Burn', 'text': 'As often as ...","[{'name': 'Fire Spin', 'cost': ['Fire', 'Fire'...",Water,3.0,...,Spits fire that is hot enough to melt boulders...,NaN,Fighting,NaN,1999-01-09,Fire Spin,"[Fire, Fire, Fire, Fire]",100,Discard 2 Energy cards attached to Charizard i...,4.0
4,base1-5,Clefairy,Basic,40.0,Colorless,NaN,NaN,"[{'name': 'Sing', 'cost': ['Colorless'], 'conv...",Fighting,1.0,...,Its magical and cute appeal has many admirers....,[Clefable],Psychic,NaN,1999-01-09,Sing,[Colorless],,"Flip a coin. If heads, the Defending Pokémon i...",1.0


In [21]:
[isinstance(item, list)for item in cards_df.attacks]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,